## Install pyodbc
Did that using conda


## Import libraries

Database details

In [15]:
import pyodbc
import pandas as pd
from IPython.display import display
from sqlalchemy import create_engine, text
import os

# Connection parameters
server_sqlserver = 'localhost'
port_sqlserver = '1433'
database_sqlserver = 'AdventureWorksDW2019'
username_sqlserver = 'sa'
password_sqlserver = 'YourStrong!Passw0rd'



server_ps = 'localhost'
port_ps = '5432'
database_ps = 'AdventureWorks'
username_ps = 'root'
password_ps = 'root'


# connection_string_sqlserver = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_sqlserver},{port_sqlserver};DATABASE={database_sqlserver};UID={username_sqlserver};PWD={password_sqlserver}' # for pyodbc

connection_string_sqlserver = f'mssql+pyodbc://{username_sqlserver}:{password_sqlserver}@{server_sqlserver}:{port_sqlserver}/{database_sqlserver}?driver=ODBC+Driver+17+for+SQL+Server'  # for sqlalchemy




# Connection string for PostgreSQL
connection_string_ps = (
    f'postgresql+psycopg2://{username_ps}:{password_ps}'
    f'@{server_ps}:{port_ps}/{database_ps}'
)


In [16]:
try:
    # engine_sqlserver = create_engine(connection_string_sqlserver)  # for sqlalchemy
    # src_conn = pyodbc.connect(connection_string_sqlserver)
    print("Connected to SqlServer successfully!\n")    
    
    engine_ps = create_engine(connection_string_ps)
    print("Connected to PostgreSQL successfully!\n")
except Exception as e:
    print("Error: ", e)

Connected to SqlServer successfully!

Connected to PostgreSQL successfully!



In [17]:
sql = text("""
            SELECT t.name AS table_name
            FROM sys.tables t 
            WHERE t.name IN ('DimProduct', 'DimProductSubCategory', 'DimProductCategory', 'DimReseller', 'DimSalesTerritory', 'FactInternetSales')
        """)


# ETL Process
- extract: get data from a source
- transform: structure, format, or clean the data
- load: write the data to an external destination

In [18]:
# creating the load function
def load(df, tbl):
    try:
        rows_imported = 0
        
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
    
        # save df to postgres
        df.to_sql(f'stg_{tbl}', engine_ps, if_exists='replace', index=False)
        print("Data imported to final print out")
    except Exception as e:
        print("Error:", str(e))
    

In [19]:
# pyodbc for which is not recommend to use therefore i have switched to sqlalchemy
# def extract():
#     try:
#         # Execute a query
#             src_cursor = src_conn.cursor()
#             src_cursor.execute(sql)
#             src_tables =src_cursor.fetchall()
# 
#             for tbl in src_tables:
#                 df2 = pd.read_sql_query(f'select * from {tbl[0]}', src_conn)
#                 load(df2,tbl[0])
#     except Exception as x:
#         print("Error: ", str(x))

In [20]:
def extract(src_conn_string,sql):
    try:
        engine = create_engine(src_conn_string)

        # Execute a query to get the list of tables
        with engine.connect() as connection:
            result = connection.execute(sql)

            src_tables = result.fetchall()

            for tbl in src_tables:
                table_name = tbl[0]

                # Load each table into a DataFrame
                df2 = pd.read_sql_query(f'SELECT * FROM {table_name}', connection)

                # Call load function to load the DataFrame
                load(df2, table_name)

    except Exception as e:
        print("Error: ", str(e))

In [21]:
try:
    extract(connection_string_sqlserver, sql)
except Exception as e:
    print("Error:", str(e))

importing rows 0 to 606... for table DimProduct
Data imported to final print out
importing rows 0 to 4... for table DimProductCategory
Data imported to final print out
importing rows 0 to 37... for table DimProductSubcategory
Data imported to final print out
importing rows 0 to 701... for table DimReseller
Data imported to final print out
importing rows 0 to 11... for table DimSalesTerritory
Data imported to final print out
importing rows 0 to 60398... for table FactInternetSales
Data imported to final print out
